## Homework 3

In this edition, I predict restuarant revenue using [ridge](https://en.wikipedia.org/wiki/Tikhonov_regularization) and  [lasso](https://en.wikipedia.org/wiki/Lasso_(statistics)) regression. Then I use [elastic](https://en.wikipedia.org/wiki/Elastic_net_regularization) regularization to 'combine' the two.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

train = pd.read_csv("Problem8_train.csv")
test = pd.read_csv("Problem8_test.csv")

In [2]:
train.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,2,3.0,4.0,3.0,1,5,5,5,5,5,...,3.0,5,5,5,3,4,4,3,1,2740687
1,3,5.0,4.0,4.0,2,5,5,4,5,4,...,2.0,0,0,0,0,0,0,0,0,5461700
2,4,5.0,4.0,4.0,2,3,5,4,4,4,...,2.0,4,1,2,2,3,4,3,2,3818055
3,3,5.0,4.0,4.0,2,2,5,1,4,4,...,2.0,5,5,3,3,3,4,3,2,6836483
4,4,5.0,5.0,4.0,1,5,5,3,4,5,...,1.0,5,5,4,3,3,4,4,1,4554237


Specify correct features, add constants to independent variables

In [4]:
x_train = train.values[:,0:37]#all values except for revenue
y_train = train[['revenue']].values #revenue
x_test = test.values[:,1:38]

x_train, x_test = sm.add_constant(x_train), sm.add_constant(x_test)

Scale the features

In [5]:
#scale paramters
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)

For the ridge regression, we try a variety of lambdas so that we get the best one

In [16]:
RR = linear_model.RidgeCV(alphas = [0.01, 0.05, 0.1, 1, 5, 10, 25, 50])
RR.fit(x_train_std, y_train)
best_lambda1 = RR.alpha_
print
print "The best lambda for Ridge Regression: ", best_lambda1
print
print"                              The R2: ", RR.score(x_train_std, y_train)
#run model on the test set
y_test_pred_ridge = RR.predict(x_test_std)


The best lambda for Ridge Regression:  50.0

                              The R2:  0.209628592221


Next up is lasso regression, which performs significantly better than Ridge regression

In [19]:
#Lasso regression on scaled data
lasso = linear_model.LassoCV(alphas = [.01, .05, .1, 1, 5, 10, 25, 50, 55, 65])
lasso.fit(x_train_std, np.ravel(y_train))
best_lambda2 = lasso.alpha_
print
print "The best lasso lambda: ", best_lambda2
print
print "               The R2: ",lasso.score(x_train_std, y_train)
print
#run it again on test set
y_test_pred_lasso = lasso.predict(x_test_std)


The best lasso lambda:  65.0

               The R2:  0.460497119868



We hit it now with elastic regularization, which also significantly detracts from the regression.

In [22]:
elastic = linear_model.ElasticNetCV(alphas = [.01, .05, .1, 1, 
                                              5, 10, 25, 50, 65])
elastic.fit(x_train_std, np.ravel(y_train))
best_lambda2 = elastic.alpha_
print
print "The best lambda: ", best_lambda2
print
print "         The R2: ", elastic.score(x_train_std, y_train)
#run it again on test set
y_test_pred_elastic = elastic.predict(x_test_std)


The best lambda:  5.0

         The R2:  0.114136377136


For comparison's sake, we'll do a regular OLS on the data.

In [25]:
ols = sm.OLS(y_train, x_train)
print ols.fit().summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     1.430
Date:                Tue, 06 Sep 2016   Prob (F-statistic):              0.105
Time:                        15:26:53   Log-Likelihood:                -1578.5
No. Observations:                 100   AIC:                             3233.
Df Residuals:                      62   BIC:                             3332.
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       6.195e+06   4.39e+06      1.411      0.1

## Conclusion

Lasso regression performs the best, and comparably to regular OLS. So there is no real advantage in this case to use Lasso over OLS.